# NifH FastANI Analysis

In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
ani_output = pd.read_csv('/vortexfs1/omics/alexander/lblum/tara_gene_finder/mag_analysis/fastani_output/nifH_vs_ref_meren', sep='\t',index_col=0, header = None)

In [3]:
ani_output.index = ani_output.index.str.replace("nifH_mags/","")
ani_output[1] = ani_output[1].str.replace("nifH_mags/","")


In [4]:
checkm_report= pd.read_csv('/vortexfs1/omics/alexander/lblum/tara_gene_finder/data/mags_data/all_checkm_plus.txt', sep='\t', index_col=0)

In [3]:
quast_report= pd.read_csv('/vortexfs1/omics/alexander/lblum/tara_gene_finder/data/mags_data/all_quast_report.txt', sep='\t', index_col=0)
quast_report = quast_report.apply(pd.to_numeric)

In [7]:
GTDBTK_report = pd.read_csv('/vortexfs1/omics/alexander/lblum/tara_gene_finder/mag_analysis/nifH-gtdbtk-2_7/gtdbtk.bac120.summary.tsv', sep='\t', index_col = 0)

In [8]:
#This is code to cluster the mags based on similarity 95% or more ANI
bins = []
index = ani_output.iloc[:,0]

for name in index:
    bins.append(name)
  
        
bin_dic = {}
used_ids = set()
for name in bins:
    if name not in bin_dic.keys() and name not in used_ids:
        bin_dic[name] = []
        clust =  ani_output[ani_output.index == name]
        
        name_list =  []
        for item in clust[1]:
            name_list.append(item)
            
        val_list = []
        for item in clust[2]:
            val_list.append(item)
    
        for val in val_list:            
            val_index= val_list.index(val)    
            if val > 95 and name_list[val_index] not in bin_dic.items():
                bin_dic[name].append(name_list[val_index])
                used_ids.add(name_list[val_index])


In [9]:
cluster_df = pd.DataFrame.from_dict(bin_dic, orient='index')
cluster_df['cluster'] = cluster_df[cluster_df.columns[0:]].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)
cluster_data = pd.DataFrame(cluster_df['cluster'])
bin_id = cluster_df.index.str.replace(".fa", "")
cluster_df["bin_id"] = bin_id
cluster_df['quast_id'] = bin_id.str.replace("-","_")

In [10]:
cluster_df = cluster_df.merge(quast_report, left_on = "quast_id", right_on = "Assembly")

In [11]:
clusters_metadata = cluster_df.merge(checkm_report, left_on = 'bin_id', right_on = "bin_id")


In [12]:
clusters_metadata = clusters_metadata.merge(GTDBTK_report, left_on = 'bin_id', right_on = "user_genome")

In [4]:
clusters_metadata_trim = clusters_metadata.loc[:, ["bin_id","cluster", "completeness", "contamination", "strain_het","classification", "Total length (>= 0 bp)", "GC (%)"]]
pd.set_option('display.max_colwidth', -1)
#here are the clusters with metadata listed for the bin_id mag (not necessarily the representative...see below)

NameError: name 'clusters_metadata' is not defined

In [41]:
full_mags = pd.DataFrame(used_ids)
bin_id = full_mags[0].str.replace("nifH_mags/","").str.replace(".fa","")
full_mags["bin_id"]= bin_id
full_mags['quast_id'] = bin_id.str.replace("-","_")
full_mags = full_mags.merge(quast_report, left_on= 'quast_id', right_on = "Assembly")
full_mags_metadata = full_mags.merge(checkm_report, left_on = 'bin_id', right_on = "bin_id")
full_mags_metadata= full_mags_metadata.merge(GTDBTK_report, left_on = 'bin_id', right_on = "user_genome")
full_mags_metadata['estimated_genome_size'] = full_mags_metadata['Total length (>= 0 bp)']/(full_mags_metadata['completeness'].astype(float)/100)
full_mags_metadata['GTDBTK_phylum'] = full_mags_metadata["classification"].str.split(";").str[1].str.replace("p__",'')
full_mags_metadata['GTDBTK_genus'] = full_mags_metadata["classification"].str.split(";").str[-2].str.replace('g__',' ')
full_mags_metadata['GTDBTK_class'] = full_mags_metadata["classification"].str.split(";").str[2].str.replace('c__','')

mags_metadata_trim = full_mags_metadata.loc[:, ["bin_id", "completeness", "contamination", "strain_het", "classification",'GTDBTK_phylum','GTDBTK_class','GTDBTK_genus', "Total length (>= 0 bp)","estimated_genome_size" ,"GC (%)"]]

##NOTE: this limits it to the MAGs from our data set only... because that is what we have metadata for. 
#This is for cross referencing clusters to choose a representative
mags_metadata_trim.to_csv("/vortexfs1/omics/alexander/lblum/tara_gene_finder/mag_analysis/outputs/nifH_brief_mags_metadata.csv", index=False)